In [1]:
import string
import pandas as pd

In [42]:
# Carrega dataframe
# Cada linha é uma proposicao
# A coluna id é o código único no sistema interno da Câmara dos Deputados
# Popularmente as proposicoes sao conhecidas pela juncao de: siglaTipo + numero + / + ano (ex. PL 2342/2010)
# Entao vou criar um campo artificial para referencia, com mais o id -> codigo_unico

In [64]:
proposicoes = pd.read_csv("resultados/projetos_camara_2010_2018.csv",\
                          sep=',',\
                          encoding = 'utf-8', \
                          converters={'ano': lambda x: str(x),
                                      'id': lambda x: str(x), 
                                      'idTipo': lambda x: str(x),
                                      'numero': lambda x: str(x)
                                      })

In [65]:
proposicoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146322 entries, 0 to 146321
Data columns (total 7 columns):
ano          146322 non-null object
ementa       144347 non-null object
id           146322 non-null object
idTipo       146322 non-null object
numero       146322 non-null object
siglaTipo    146321 non-null object
uri          146320 non-null object
dtypes: object(7)
memory usage: 7.8+ MB


In [66]:
def cria(linha):
    tipo = str(linha["siglaTipo"])
    numero = str(linha['numero'])
    ano = str(linha['ano'])
    id_ = str(linha['id'])
    documento = tipo + ' ' + numero + '/' + ano + ' - ' + id_
    return documento

proposicoes["codigo_unico"] = proposicoes.apply(cria, axis=1)

In [67]:
# Trabalha com uma copia da coluna ementa

In [68]:
proposicoes["ementa_copia"] = proposicoes["ementa"]

In [69]:
proposicoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146322 entries, 0 to 146321
Data columns (total 9 columns):
ano             146322 non-null object
ementa          144347 non-null object
id              146322 non-null object
idTipo          146322 non-null object
numero          146322 non-null object
siglaTipo       146321 non-null object
uri             146320 non-null object
codigo_unico    146322 non-null object
ementa_copia    144347 non-null object
dtypes: object(9)
memory usage: 10.0+ MB


In [70]:
# Rotinas para tirar pontuacao e deixar minuscula

In [71]:
def limpeza1(linha):
    documento = str(linha["ementa_copia"])
    documento = documento.strip()
    documento = documento.replace(',', '')
    documento = documento.replace('.', '')
    documento = documento.replace('-', '')
    documento = documento.replace('/', '')
    documento = documento.replace(';', '')
    documento = documento.replace('"', '')
    documento = documento.translate(string.punctuation)
    documento = documento.lower()
    return documento
    
proposicoes["ementa_copia"] = proposicoes.apply(limpeza1, axis=1)

In [72]:
# Cria arquivo para checagem visual

In [73]:
proposicoes.to_excel('resultados/teste.xlsx',sheet_name='Sheet1')

In [74]:
proposicoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146322 entries, 0 to 146321
Data columns (total 9 columns):
ano             146322 non-null object
ementa          144347 non-null object
id              146322 non-null object
idTipo          146322 non-null object
numero          146322 non-null object
siglaTipo       146321 non-null object
uri             146320 non-null object
codigo_unico    146322 non-null object
ementa_copia    146322 non-null object
dtypes: object(9)
memory usage: 10.0+ MB


In [ ]:
#Words_array

In [75]:
def limpeza2(linha):
    documento = str(linha["ementa_copia"])
    documento = documento.split(' ')
    return documento
      
proposicoes["words_array"] = proposicoes.apply(limpeza2, axis=1)

In [76]:
proposicoes.reset_index().head()

,index,ano,ementa,id,idTipo,numero,siglaTipo,uri,codigo_unico,ementa_copia,words_array
0,0,2010,Comunica o Excelentíssimo Senhor Presidente da...,465465,412,36,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...,MSC 36/2010 - 465465,comunica o excelentíssimo senhor presidente da...,"[comunica, o, excelentíssimo, senhor, presiden..."
1,1,2010,"Abre crédito extraordinário, em favor da Presi...",465466,291,480,MPV,https://dadosabertos.camara.leg.br/api/v2/prop...,MPV 480/2010 - 465466,abre crédito extraordinário em favor da presid...,"[abre, crédito, extraordinário, em, favor, da,..."
2,2,2010,Comunica o Excelentíssimo Senhor Presidente da...,465467,415,38,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...,MSC 38/2010 - 465467,comunica o excelentíssimo senhor presidente da...,"[comunica, o, excelentíssimo, senhor, presiden..."
3,3,2010,Submete à deliberação do Congresso Nacional Pr...,465469,134,42,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...,MSC 42/2010 - 465469,submete à deliberação do congresso nacional pr...,"[submete, à, deliberação, do, congresso, nacio..."
4,4,2010,Concede auxílio especial e bolsa especial aos ...,465470,139,6720,PL,https://dadosabertos.camara.leg.br/api/v2/prop...,PL 6720/2010 - 465470,concede auxílio especial e bolsa especial aos ...,"[concede, auxílio, especial, e, bolsa, especia..."
